In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, average_precision_score, precision_recall_curve, confusion_matrix, plot_roc_curve, roc_auc_score, classification_report

import feather
import scikitplot as skplt
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from hpsklearn import HyperoptEstimator, any_classifier
from hyperopt import tpe
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.pipeline import Pipeline

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [2]:
PATH='/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/tmp/df_ft.feather'

In [3]:
df_ft = feather.read_dataframe(PATH)

correlations = df_ft.corr()
correlations = correlations['URLType']
display(correlations)

robust = RobustScaler()

x = df_ft.drop('URLType', axis=1)
y = df_ft["URLType"]

# x_scaled = preprocessing.scale(x)
# display(len(x_scaled))
# display(len(y))
# # x = x.reset_index()
# # # y = y.reset_index()
# # np.where(x.values >= np.finfo(np.float64).max)
# test_size = int(0.2 * len(y))
# np.random.seed(13)
# indices = np.random.permutation(len(x))

# display(indices)
# X_train = x[indices[:-test_size]]
# x_test = x[indices[-test_size:]]
# Y_train = y[indices[:-test_size]]
# y_test = y[indices[-test_size:]]

X_train, x_test, Y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
display(X_train)

for (column_name, column_data) in X_train.iteritems():
    X_train[column_name] = robust.fit_transform(X_train[column_name].values.reshape(-1, 1))
    
display(X_train)

for (column_name, column_data) in x_test.iteritems():
    x_test[column_name] = robust.fit_transform(x_test[column_name].values.reshape(-1, 1))
    
display(x_test)
# X_train = robust.fit_transform(X_train.values.reshape(-1, 1))
# x_test = robust.fit_transform(x_test.values.reshape(-1, 1))

# display(X_train)
# display(x_test)
# # np.where(X_train >= np.finfo(np.float64).max)

# display(X_train)

URLLength         -0.297791
HostLength         0.171448
TLDLength               NaN
DotCount           0.068579
DashCount         -0.454538
@Count            -0.027748
%Count             0.086102
=Count             0.064490
?Count             0.031363
DigitCount         0.020819
UniqueCharCount   -0.297085
URLType            1.000000
Name: URLType, dtype: float64

,URLLength,HostLength,TLDLength,DotCount,DashCount,@Count,%Count,=Count,?Count,DigitCount,UniqueCharCount
41530,129,13,0,2,17,0,0,0,0,6,28
2232,108,7,0,1,14,0,0,0,0,6,33
40712,96,14,0,1,8,0,0,0,0,10,32
40360,102,14,0,1,11,0,0,0,0,7,29
9908,97,13,0,1,12,0,0,0,0,6,30
...,...,...,...,...,...,...,...,...,...,...,...
11284,71,14,0,1,5,0,0,0,0,1,24
44732,116,10,0,1,7,1,0,1,1,5,32
38158,128,13,0,1,15,0,0,0,0,0,24
860,93,11,0,1,15,0,0,0,0,0,23


/Users/chrismclearnon/miniconda3/envs/proj-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrismclearnon/miniconda3/envs/proj-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrismclearnon/miniconda3/envs/proj-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,URLLength,HostLength,TLDLength,DotCount,DashCount,@Count,%Count,=Count,?Count,DigitCount,UniqueCharCount
41530,1.269231,0.166667,0.0,1.0,1.444444,0.0,0.0,0.0,0.0,-0.176471,-0.285714
2232,0.461538,-0.833333,0.0,0.0,1.111111,0.0,0.0,0.0,0.0,-0.176471,0.428571
40712,0.000000,0.333333,0.0,0.0,0.444444,0.0,0.0,0.0,0.0,0.058824,0.285714
40360,0.230769,0.333333,0.0,0.0,0.777778,0.0,0.0,0.0,0.0,-0.117647,-0.142857
9908,0.038462,0.166667,0.0,0.0,0.888889,0.0,0.0,0.0,0.0,-0.176471,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
11284,-0.961538,0.333333,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,-0.470588,-0.857143
44732,0.769231,-0.333333,0.0,0.0,0.333333,1.0,0.0,1.0,1.0,-0.235294,0.285714
38158,1.230769,0.166667,0.0,0.0,1.222222,0.0,0.0,0.0,0.0,-0.529412,-0.857143
860,-0.115385,-0.166667,0.0,0.0,1.222222,0.0,0.0,0.0,0.0,-0.529412,-1.000000


/Users/chrismclearnon/miniconda3/envs/proj-env/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrismclearnon/miniconda3/envs/proj-env/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrismclearnon/miniconda3/envs/proj-env/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,URLLength,HostLength,TLDLength,DotCount,DashCount,@Count,%Count,=Count,?Count,DigitCount,UniqueCharCount
8961,0.222222,-0.500000,0.0,1.0,1.000000,0.0,0.0,0.0,0.0,0.6250,-0.142857
19497,-0.074074,0.333333,0.0,1.0,0.222222,0.0,0.0,0.0,0.0,-0.0625,0.000000
41378,-0.740741,1.500000,0.0,3.0,-0.444444,0.0,0.0,0.0,0.0,-0.5000,-0.714286
23467,-0.333333,0.500000,0.0,0.0,-0.444444,0.0,0.0,0.0,0.0,1.0000,0.000000
20788,-0.074074,-1.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,-0.1250,-0.142857
...,...,...,...,...,...,...,...,...,...,...,...
27804,0.703704,1.333333,0.0,0.0,0.555556,0.0,0.0,0.0,0.0,-0.1250,-0.285714
33659,-0.259259,-0.333333,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,-0.1250,0.285714
41524,0.111111,-0.666667,0.0,0.0,0.888889,0.0,0.0,0.0,0.0,-0.5000,-0.857143
26032,0.370370,0.500000,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,-0.0625,-0.142857


In [4]:
pipe = Pipeline([
    ('model', SVC())
])

svc_search = {
    'model': Categorical([SVC()]),
    'model__C': Real(0.01, 100.0, 'log-uniform'),
#     'model__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
#     'model__degree': Integer(1,8),
    'model__kernel': Categorical(['poly', 'rbf']),
}

knn_search = {
    'model': Categorical([KNeighborsClassifier()]),
    'model__n_neighbours': Integer(1, 10),
    'model__algorithm': 'auto',
}

searchcv = BayesSearchCV(
    pipe,
    search_spaces = svc_search, # (parameter space, # of evaluations)
#     cv=3
#     SVC(gamma='scale'),
#     search_spaces={'C': (0.01, 100.0, 'log-uniform')},
    n_iter=10,
    cv=3,
    verbose=5,
    n_jobs=-1
)



In [5]:
def on_step(optim_result):
    score = searchcv.best_score_
    print("best score: %s" % score)
    if score >= 0.98:
        print('Interrupting!')
        return True
    
print(searchcv.total_iterations)

30


In [6]:
import time
start_time = time.time()

searchcv.fit(X_train, Y_train, callback=on_step)

print("--- %s seconds ---" % (time.time() - start_time))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   13.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   13.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


best score: 0.9538543469577953
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   15.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   15.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


best score: 0.9538543469577953
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

best score: 0.9557182798562109
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   28.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   28.0s finished


best score: 0.9557182798562109
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   29.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   29.5s finished


best score: 0.9557182798562109
--- 320.70079016685486 seconds ---


In [8]:
searchcv.score(x_test, y_test)

0.9553733091916072